# Исследование надёжности заёмщиков

Заказчик — кредитный отдел банка. Нужно разобраться, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок. Входные данные от банка — статистика о платёжеспособности клиентов.

Результаты исследования будут учтены при построении модели **кредитного скоринга** — специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку.

## Шаг 1. Откройте файл с данными и изучите общую информацию

Импортируем библиотеку pandas, связав вызов библиотеки с коротким именем pd, посмотрим на общую информацию об исходных данных и взглянем на первые строки:

In [1]:
import pandas as pd

data = pd.read_csv('/datasets/data.csv')
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       19351 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        19351 non-null float64
purpose             21525 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


In [2]:
data.head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу


В глаза сразу бросаются аномалии в виде отрицательных значений в столбце "days_employed". Напишем и применим функцию для проверки столбцов на подобные вещи:

In [3]:
def negative_count(data):
    negative_count = data.select_dtypes(include=['float64', 'int64'])
    negative_count = negative_count[negative_count < 0].count()
    return negative_count #функция возвращает количество отрицательных значений в каждом столбце передаваемой таблицы

negative_count(data)

children               47
days_employed       15906
dob_years               0
education_id            0
family_status_id        0
debt                    0
total_income            0
dtype: int64

Проверим на странности минимальные и максимальные значения столбцов с помощью функции:

In [4]:
'''def min_max(data):
    dict = {}
    min_max_table = data.select_dtypes(include=['float64', 'int64'])
    for row in min_max_table:
        dict[row] = [data[row].min(), data[row].max()]
        min_max_table = pd.DataFrame.from_dict(dict)
    return min_max_table #возвращает таблицу с минимальным и максимальным значением каждого столбца типа данных int и float

min_max(data)'''

data.describe()

,children,days_employed,dob_years,education_id,family_status_id,debt,total_income
count,21525.000000,19351.000000,21525.000000,21525.000000,21525.000000,21525.000000,1.935100e+04
mean,0.538908,63046.497661,43.293380,0.817236,0.972544,0.080883,1.674223e+05
std,1.381587,140827.311974,12.574584,0.548138,1.420324,0.272661,1.029716e+05
min,-1.000000,-18388.949901,0.000000,0.000000,0.000000,0.000000,2.066726e+04
25%,0.000000,-2747.423625,33.000000,1.000000,0.000000,0.000000,1.030532e+05
50%,0.000000,-1203.369529,42.000000,1.000000,0.000000,0.000000,1.450179e+05
75%,1.000000,-291.095954,53.000000,1.000000,1.000000,0.000000,2.034351e+05
max,20.000000,401755.400475,75.000000,4.000000,4.000000,1.000000,2.265604e+06


Отрицательное количество детей следует исправить в дальнейшем.

Напишем и применим функцию для выявления уникальных значениях в столбцах с типом данных "object":

In [5]:
def table_of_unique(data):
    data_filtered = data.select_dtypes(include=['object'])
    dict = {}
    for row in data_filtered:
        dict[row] = data_filtered[row].unique().tolist()
    table_of_unique = pd.DataFrame.from_dict(dict, orient='index').transpose()
    return table_of_unique

table_of_unique(data)

,education,family_status,gender,income_type,purpose
0,высшее,женат / замужем,F,сотрудник,покупка жилья
1,среднее,гражданский брак,M,пенсионер,приобретение автомобиля
2,Среднее,вдовец / вдова,XNA,компаньон,дополнительное образование
3,СРЕДНЕЕ,в разводе,None,госслужащий,сыграть свадьбу
4,ВЫСШЕЕ,Не женат / не замужем,None,безработный,операции с жильем
5,неоконченное высшее,None,None,предприниматель,образование
6,начальное,None,None,студент,на проведение свадьбы
7,Высшее,None,None,в декрете,покупка жилья для семьи
8,НЕОКОНЧЕННОЕ ВЫСШЕЕ,None,None,None,покупка недвижимости
9,Неоконченное высшее,None,None,None,покупка коммерческой недвижимости


Проверим корректность присвоения id в столбцах со сведениями об образовании и семейном статусе клиентов:

In [6]:
data.groupby('education_id')['education'].unique()

education_id
0                             [высшее, ВЫСШЕЕ, Высшее]
1                          [среднее, Среднее, СРЕДНЕЕ]
2    [неоконченное высшее, НЕОКОНЧЕННОЕ ВЫСШЕЕ, Нео...
3                    [начальное, НАЧАЛЬНОЕ, Начальное]
4     [Ученая степень, УЧЕНАЯ СТЕПЕНЬ, ученая степень]
Name: education, dtype: object

In [7]:
data.groupby('family_status_id')['family_status'].unique()

family_status_id
0          [женат / замужем]
1         [гражданский брак]
2           [вдовец / вдова]
3                [в разводе]
4    [Не женат / не замужем]
Name: family_status, dtype: object

Ошибок нет, id присвоены корректно.

**Вывод**

В исходных данных наблюдаются следующие проблемы:
- отрицательные значения в столбцах "children" и "days_employed"
- аномалии в столбце "days_employed" в виде отрицательных и слишком больших значений
- неявные дубликаты в столбцах "education" и "purpose"
- пропущены значения в столбцах "total_income" и "days_employed"
- нулевые значения в возрасте заемщиков

## Шаг 2. Предобработка данных

### Обработка пропусков

Перед тем как приступить к устранению неявных пропусков в виде нулевого возраста у заёмщиков, оценим масштаб проблемы:

In [8]:
print(sorted(data['dob_years'].unique()))
# без print юпитер выводит в очень некрасивом виде

[0, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75]


In [9]:
print(f'Количество заёмщиков с возрастом 0 лет: {len(data.loc[data["dob_years"] == 0])}')

Количество заёмщиков с возрастом 0 лет: 101


По типам занятости это аномалия распределяется так:

In [10]:
data.loc[data['dob_years'] == 0, 'income_type'].value_counts()

сотрудник      55
пенсионер      20
компаньон      20
госслужащий     6
Name: income_type, dtype: int64

Следует аккуратно обойтись с этим неявным пропуском и заменить его на медианный возраст для каждого типа занятости, чтобы не искажать картину результаты исследования. Вычислим медиану для каждого типа занятости:

In [11]:
median_dob_years = data.groupby('income_type')['dob_years'].median()
median_dob_years

income_type
безработный        38.0
в декрете          39.0
госслужащий        40.0
компаньон          39.0
пенсионер          60.0
предприниматель    42.5
сотрудник          39.0
студент            22.0
Name: dob_years, dtype: float64

Теперь применим цикл для замены нулевых значений:

In [12]:
for i in data['income_type'].unique():
    data.loc[(data['dob_years'] == 0) & (data['income_type'] == i), 'dob_years'] = median_dob_years[i]

Проверка:

In [13]:
data.loc[data['dob_years'] == 0, 'income_type'].value_counts()

Series([], Name: income_type, dtype: int64)

Перед тем как перейти к устранению пропусков в столбцах "total_income" и "days_employed" взглянем на данные:

In [14]:
data[data['total_income'].isna()].head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
12,0,NaN,65.0,среднее,1,гражданский брак,1,M,пенсионер,0,NaN,сыграть свадьбу
26,0,NaN,41.0,среднее,1,женат / замужем,0,M,госслужащий,0,NaN,образование
29,0,NaN,63.0,среднее,1,Не женат / не замужем,4,F,пенсионер,0,NaN,строительство жилой недвижимости
41,0,NaN,50.0,среднее,1,женат / замужем,0,F,госслужащий,0,NaN,сделка с подержанным автомобилем
55,0,NaN,54.0,среднее,1,гражданский брак,1,F,пенсионер,1,NaN,сыграть свадьбу


Количество пропусков в столбце "total_income":

In [15]:
data['total_income'].isna().sum()

2174

Количество пропусков в столбце "days_employed":

In [16]:
data['days_employed'].isna().sum()

2174

Проверим предположение о том, что пропуски в сведениях о доходах связаны с отсутствием сведений о трудовом стаже:

In [17]:
len(data.loc[data['days_employed'].isna() & data['total_income'].isna()])

2174

По типу занятости пропуски группируются так:

In [18]:
data[data['total_income'].isna()]['income_type'].value_counts()

сотрудник          1105
компаньон           508
пенсионер           413
госслужащий         147
предприниматель       1
Name: income_type, dtype: int64

Предположение подтвердилось. К тому же, эти сведения отсутствуют почти по каждому десятому клиенту. Следует заменить значения в соответствующих столбцах на медиану, т.к. принятие их за 0 может привести к искажению результатов исследования.

Найдем медианный доход для каждого типа занятости:

In [19]:
median_income = data.groupby('income_type')['total_income'].median()
median_income

income_type
безработный        131339.751676
в декрете           53829.130729
госслужащий        150447.935283
компаньон          172357.950966
пенсионер          118514.486412
предприниматель    499163.144947
сотрудник          142594.396847
студент             98201.625314
Name: total_income, dtype: float64

Отметим, что сведения о доходах и стаже не пропущены у безработных, находящихся в декрете и студентов.
<br>
Теперь пора заполнить пропуски в доходах на медиану по типам занятости:

In [20]:
for i in data['income_type'].unique():
    data.loc[((data['total_income'].isna()) & (data['income_type'] == i)), 'total_income'] = median_income[i]


Проверим как устранились пропущенные значения доходов:

In [21]:
data[data['total_income'].isna()]['income_type'].value_counts()

Series([], Name: income_type, dtype: int64)

Вернемся к пропущенным значениям стажа:

In [22]:
data[data['days_employed'].isna()]['income_type'].value_counts()

сотрудник          1105
компаньон           508
пенсионер           413
госслужащий         147
предприниматель       1
Name: income_type, dtype: int64

Ранее были отмечены странности в виде отрицательного или слишком большого стажа, возможно эти аномалии не случайны.
<br>
Чтобы не терять важную информацию, заменим пропущенный стаж на медианые значения для возрастных групп. Для начала следует добавить столбец с id этих групп в таблицу по следующим критериям:
- первая группа: молодые заёмщики в возрасте до 35 лет включительно
- вторая группа: зрелые заёмщики в возрасте от 35 до 59 лет включительно
- третья группа: пожилые заёмщики в возрасте от 60 лет

In [23]:
def dob_years_id(dob_years):
    if dob_years <= 35:
        return 0
    elif 35 < dob_years < 60:
        return 1
    elif dob_years >= 60:
        return 2
data['dob_years_id'] = data['dob_years'].apply(dob_years_id)

Посчитаем медиану для каждой возрастной группы:

In [24]:
median_days_employed = data.groupby('dob_years_id')['days_employed'].median()
median_days_employed

dob_years_id
0     -1197.741206
1     -1584.671417
2    355324.823353
Name: days_employed, dtype: float64

Заменим пропуски на медиану и проверим результат:

In [25]:
for i in data['dob_years_id'].unique():
    data.loc[(data['days_employed'].isna()) & (data['dob_years_id'] == i), 'days_employed'] = median_days_employed[i]
data[data['days_employed'].isna()]['income_type'].value_counts()
#можно проще через data['days_employed'].isna().sum(), но я скопипастил предыдущий код, которым считал пропуски по занятости

Series([], Name: income_type, dtype: int64)

С заменой пропусков справились. Посмотрим на проблему отрицательного количества детей:

In [26]:
negative_kids_share = len(data.loc[data['children'] == -1, 'children']) / len(data)
no_kids_share = len(data.loc[data['children'] == 0, 'children']) / len(data)
one_kid_share = len(data.loc[data['children'] == 1, 'children']) / len(data)
print(f'Процент заёмщиков с -1 рёбенком: {negative_kids_share:.2%}')
print(f'Процент заёмщиков с 0 рёбенком: {no_kids_share:.2%}')
print(f'Процент заёмщиков с 1 рёбенком: {one_kid_share:.2%}')

Процент заёмщиков с -1 рёбенком: 0.22%
Процент заёмщиков с 0 рёбенком: 65.73%
Процент заёмщиков с 1 рёбенком: 22.38%


Незначительный процент заёмщиков оказался с -1 ребёнком. Скорее всего имеет место проблема сбора данных, чтобы их не терять, предположим, что на самом деле у этих клиентов банка 1 ребёнок:

In [27]:
#data.loc[data['children'] == -1, 'children'] = 1
data['children'] = data['children'].abs()
data.loc[data['children'] == -1, 'children']

Series([], Name: children, dtype: int64)

Аномалия устранена!

**Вывод**

Сведения о стаже и доходах пропущены только у сотрудников, компаньонов, пенсионеров, госслужащих и предпринимателей, т.е. у групп, которым полагается иметь реальные стаж и доход. Возможно это проблема сбора данных, либо последствия неофициальной занятости. Примерно у той же группы (исключая предпринимателей, что неудивительно, т.к. их мало) в сведениях о возрасте внесено значение 0.
<br>
Что касается странных значений в стаже, можно предположить что в соответствующем столбце указан не общий стаж, а отклонение от дельты.

### Замена типа данных

Приводим в порядок стобец "total_income". Учитывая бесполезность учёта доходов с точностью в 6 знаков после запятой, следует присвоить значениям в столбце целочисленный тип для удобства дальнейшей обработки и восприятия:

In [28]:
data['total_income'] = data['total_income'].astype('int64')

In [29]:
data.head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,dob_years_id
0,1,-8437.673028,42.0,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья,1
1,1,-4024.803754,36.0,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля,1
2,0,-5623.422610,33.0,Среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья,0
3,3,-4124.747207,32.0,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование,0
4,0,340266.072047,53.0,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу,1


**Вывод**

Значения столбца "total_income" выглядят красиво и опрятно. Ура!

### Обработка дубликатов

Переходим к устранению явных дубликатов. Значение в столбце "education" выглядят так:

In [30]:
data.groupby('education_id')['education'].unique()

education_id
0                             [высшее, ВЫСШЕЕ, Высшее]
1                          [среднее, Среднее, СРЕДНЕЕ]
2    [неоконченное высшее, НЕОКОНЧЕННОЕ ВЫСШЕЕ, Нео...
3                    [начальное, НАЧАЛЬНОЕ, Начальное]
4     [Ученая степень, УЧЕНАЯ СТЕПЕНЬ, ученая степень]
Name: education, dtype: object

Очевидно, что ввод сведений об образовании осуществляется вручную. Стоит подумать о выборе из готовых вариантов. Приведем написание в нижний регистр:

In [31]:
data['education'] = data['education'].str.lower()
data.groupby('education_id')['education'].unique()

education_id
0                 [высшее]
1                [среднее]
2    [неоконченное высшее]
3              [начальное]
4         [ученая степень]
Name: education, dtype: object

Оцениваем ситуацию с дубликатами:

In [32]:
data[data.duplicated(keep=False)].sort_values(by=['days_employed', 'dob_years', 'purpose', 'total_income']).head(10)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,dob_years_id
9013,2,-1584.671417,36.0,высшее,0,женат / замужем,0,F,госслужащий,0,150447,получение образования,1
14432,2,-1584.671417,36.0,высшее,0,женат / замужем,0,F,госслужащий,0,150447,получение образования,1
8886,1,-1584.671417,37.0,среднее,1,женат / замужем,0,F,сотрудник,0,142594,покупка недвижимости,1
12375,1,-1584.671417,37.0,среднее,1,женат / замужем,0,F,сотрудник,0,142594,покупка недвижимости,1
9374,0,-1584.671417,38.0,высшее,0,гражданский брак,1,F,компаньон,0,172357,на проведение свадьбы,1
19387,0,-1584.671417,38.0,высшее,0,гражданский брак,1,F,компаньон,0,172357,на проведение свадьбы,1
11033,2,-1584.671417,39.0,среднее,1,гражданский брак,1,F,сотрудник,0,142594,сыграть свадьбу,1
16902,2,-1584.671417,39.0,среднее,1,гражданский брак,1,F,сотрудник,0,142594,сыграть свадьбу,1
4081,1,-1584.671417,40.0,среднее,1,гражданский брак,1,F,компаньон,0,172357,строительство жилой недвижимости,1
17774,1,-1584.671417,40.0,среднее,1,гражданский брак,1,F,компаньон,0,172357,строительство жилой недвижимости,1


In [33]:
print(f'Дубликаты есть и их много! А именно {len(data[data.duplicated()])}.')

Дубликаты есть и их много! А именно 71.


Избавимся от них методом drop_duplicates и проверим результат:

In [34]:
data.drop_duplicates(inplace=True)
data.reset_index(drop=True, inplace=True)
print(f'Осталось {len(data[data.duplicated()])} дубликатов.')

Осталось 0 дубликатов.


**Вывод**

После приведения значений столбца 'education' в нижний регистр в таблице оказалась 71 дублирующая строка. Следует уточнить причину появления дубликатов, возможно это могут быть повторные заявки, либо два кредита на схожие цели. Так как количество дубликатов незначительно, их можно удалить без потери ценных сведений.

### Лемматизация

Поиграемся с выкинутой из новой версии проекта лемматизацией, вместо того чтобы передалать весь ноутбук под обновившееся задание:

In [35]:
from pymystem3 import Mystem
from collections import Counter
m = Mystem()

def lemmatize(string):
    lemmas = m.lemmatize(string)
    return lemmas
data['purpose_lemmas'] = data['purpose'].apply(lemmatize)
data.head(10)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,dob_years_id,purpose_lemmas
0,1,-8437.673028,42.0,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья,1,"[покупка, , жилье, \n]"
1,1,-4024.803754,36.0,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля,1,"[приобретение, , автомобиль, \n]"
2,0,-5623.422610,33.0,среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья,0,"[покупка, , жилье, \n]"
3,3,-4124.747207,32.0,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование,0,"[дополнительный, , образование, \n]"
4,0,340266.072047,53.0,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу,1,"[сыграть, , свадьба, \n]"
5,0,-926.185831,27.0,высшее,0,гражданский брак,1,M,компаньон,0,255763,покупка жилья,0,"[покупка, , жилье, \n]"
6,0,-2879.202052,43.0,высшее,0,женат / замужем,0,F,компаньон,0,240525,операции с жильем,1,"[операция, , с, , жилье, \n]"
7,0,-152.779569,50.0,среднее,1,женат / замужем,0,M,сотрудник,0,135823,образование,1,"[образование, \n]"
8,2,-6929.865299,35.0,высшее,0,гражданский брак,1,F,сотрудник,0,95856,на проведение свадьбы,0,"[на, , проведение, , свадьба, \n]"
9,0,-2188.756445,41.0,среднее,1,женат / замужем,0,M,сотрудник,0,144425,покупка жилья для семьи,1,"[покупка, , жилье, , для, , семья, \n]"


Получилось добавить новый столбец с леммами.

Попробуем выбросить лишние слова из столбца "purpose":

In [36]:
for i in range(len(data)):
    for e in ['\n', ' ', 'с', 'для', 'со', 'на', 'свой', 'операция']:
        while e in data['purpose_lemmas'][i]:
            data['purpose_lemmas'][i].remove(e)
        else:
            continue

In [37]:
data.head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,dob_years_id,purpose_lemmas
0,1,-8437.673028,42.0,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья,1,"[покупка, жилье]"
1,1,-4024.803754,36.0,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля,1,"[приобретение, автомобиль]"
2,0,-5623.422610,33.0,среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья,0,"[покупка, жилье]"
3,3,-4124.747207,32.0,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование,0,"[дополнительный, образование]"
4,0,340266.072047,53.0,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу,1,"[сыграть, свадьба]"


Почти получилось! Но цикл явно нуждается в доработке:

In [38]:
for i in range(len(data)):
    for e in ['\n', ' ', 'с', 'для', 'со', 'на', 'свой', 'операция', 'покупка', 'проведение', 'сыграть', 'коммерческий',
              'сдача', 'семья', 'собственный', 'строительство', 'жилой', 'сделка', 'поддержать', 'заниматься', 'высокий',
             'высокий', 'получение', 'дополнительный', 'ремонт', 'подержанный', 'приобретение', 'подержать', 'профильный']:
        while e in data['purpose_lemmas'][i]:
            data['purpose_lemmas'][i].remove(e)
        else:
            continue

Выведем таблицу с категориями займов:

In [39]:
top_purposes = pd.DataFrame(data.loc[:, 'purpose_lemmas'].value_counts().reset_index())
top_purposes.rename(columns={'index': 'purpose_type', 'purpose_lemmas':'sum_by_purpose'}, inplace=True)
top_purposes

,purpose_type,sum_by_purpose
0,[недвижимость],6351
1,[жилье],4460
2,[автомобиль],4306
3,[образование],4013
4,[свадьба],2324


**Вывод**

Умение пользоваться лемматизацией получено, страшный цикл написан, осознание того, что можно было просто написать функцию на вхождение подстроки в строку и применить её к исходному датасету через apply пришло.

### Категоризация данных

Количественное распредление заёмщиков по целям займа мы выяснили в прошлом пункте. Сейчас же посмотрим какие доли от общего количества эти группы составляют:

In [40]:
for i in range(len(top_purposes)):
    print(f'Обращений с целью "{"".join(top_purposes.loc[i, "purpose_type"])}": {top_purposes.loc[i, "sum_by_purpose"] / len(data):.2%}')

Обращений с целью "недвижимость": 29.60%
Обращений с целью "жилье": 20.79%
Обращений с целью "автомобиль": 20.07%
Обращений с целью "образование": 18.71%
Обращений с целью "свадьба": 10.83%


Попробуем категоризировать заявителей по целям займа с помощью функции, на вход она принимает цель займа, если в цели содержится одна из найденных лемм, то функция вернет id, тут же объединим цели "недвижимость" и "жилье":

In [41]:
def purpose_id(purpose):
    if 'недвижимость' in purpose:
        return 0
    elif 'жилье' in purpose:
        return 0
    elif 'автомобиль' in purpose:
        return 1
    elif 'образование' in purpose:
        return 2
    elif 'свадьба' in purpose:
        return 3

Пора применить и проверить работу функции:

In [42]:
data['purpose_id'] = data['purpose_lemmas'].apply(purpose_id)
data.head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,dob_years_id,purpose_lemmas,purpose_id
0,1,-8437.673028,42.0,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья,1,[жилье],0
1,1,-4024.803754,36.0,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля,1,[автомобиль],1
2,0,-5623.422610,33.0,среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья,0,[жилье],0
3,3,-4124.747207,32.0,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование,0,[образование],2
4,0,340266.072047,53.0,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу,1,[свадьба],3


Посмотрим, что поменялось после объединения целей "недвижимость"и "жилье":

In [43]:
print(f'Обращений с целью "недвижимость": {data.loc[data["purpose_id"] == 0, "purpose_id"].count() / len(data):.2%}')
print(f'Обращений с целью "автомобиль": {data.loc[data["purpose_id"] == 1, "purpose_id"].count() / len(data):.2%}')
print(f'Обращений с целью "образование": {data.loc[data["purpose_id"] == 2, "purpose_id"].count() / len(data):.2%}')
print(f'Обращений с целью "свадьба": {data.loc[data["purpose_id"] == 3, "purpose_id"].count() / len(data):.2%}')

Обращений с целью "недвижимость": 50.39%
Обращений с целью "автомобиль": 20.07%
Обращений с целью "образование": 18.71%
Обращений с целью "свадьба": 10.83%


Леммы свою роль сыграли, пора от них избавиться:

In [44]:
data.drop('purpose_lemmas', axis=1, inplace=True)
data.head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,dob_years_id,purpose_id
0,1,-8437.673028,42.0,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья,1,0
1,1,-4024.803754,36.0,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля,1,1
2,0,-5623.422610,33.0,среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья,0,0
3,3,-4124.747207,32.0,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование,0,2
4,0,340266.072047,53.0,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу,1,3


Для категоризации заёмщиков по уровню образования попробуем обойтись без написания функции. Вместо этого создадим таблицу в которой будет посчитано количество заявителей с каждым уровнем образования:

In [45]:
top_education = pd.DataFrame(data['education'].value_counts().reset_index())
top_education.rename(columns={'index': 'education_type', 'education':'sum_by_education'}, inplace=True)
top_education

,education_type,sum_by_education
0,среднее,15172
1,высшее,5250
2,неоконченное высшее,744
3,начальное,282
4,ученая степень,6


Красота! Посчитаем доли:

In [46]:
for i in range(len(top_education)):
    print(f'Заёмщиков с образованием "{top_education.loc[i, "education_type"]}": {top_education.loc[i, "sum_by_education"] / len(data):.2%}')

Заёмщиков с образованием "среднее": 70.72%
Заёмщиков с образованием "высшее": 24.47%
Заёмщиков с образованием "неоконченное высшее": 3.47%
Заёмщиков с образованием "начальное": 1.31%
Заёмщиков с образованием "ученая степень": 0.03%


Подобным образом узнаем как представлены заёмщики с разными семейными статусами:

In [47]:
top_family_status = pd.DataFrame(data['family_status'].value_counts().reset_index())
top_family_status.rename(columns={'index': 'family_type', 'family_status':'sum_by_family'}, inplace=True)
top_family_status

,family_type,sum_by_family
0,женат / замужем,12339
1,гражданский брак,4151
2,Не женат / не замужем,2810
3,в разводе,1195
4,вдовец / вдова,959


In [48]:
for i in range(len(data.groupby('family_status_id')['family_status'].unique())):
    print(f'Заёмщиков с семейным положением "{top_family_status.loc[i, "family_type"]}": {top_family_status.loc[i, "sum_by_family"] / len(data):.2%}')

Заёмщиков с семейным положением "женат / замужем": 57.51%
Заёмщиков с семейным положением "гражданский брак": 19.35%
Заёмщиков с семейным положением "Не женат / не замужем": 13.10%
Заёмщиков с семейным положением "в разводе": 5.57%
Заёмщиков с семейным положением "вдовец / вдова": 4.47%


Добавим в исходную таблицу категоризацию по наличию детей в семье:

In [49]:
def children_id(children):
    if children >= 1:
        return 1
    else:
        return 0
data['children_id'] = data['children'].apply(children_id).astype('int64')
data.head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,dob_years_id,purpose_id,children_id
0,1,-8437.673028,42.0,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья,1,0,1
1,1,-4024.803754,36.0,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля,1,1,1
2,0,-5623.422610,33.0,среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья,0,0,0
3,3,-4124.747207,32.0,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование,0,2,1
4,0,340266.072047,53.0,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу,1,3,0


Посмотрим на доли заёмщиков с детьми и без детей:

In [50]:
print(f'Заёмщиков с детьми: {data.loc[data["children_id"] == 1, "children_id"].count() / len(data):.2%}')
print(f'Заёмщиков без детей: {data.loc[data["children_id"] == 0, "children_id"].count() / len(data):.2%}')

Заёмщиков с детьми: 34.32%
Заёмщиков без детей: 65.68%


Категоризируем заёмщиков по доходам. Очевидный способ - выделить квартили и добавить в исходную таблицу столбец "total_income_id". Сначала применим методом describe() к исходным данным о доходе заёмщиков и сохраним результат в переменную "total_income_described":

In [51]:
total_income_described = data['total_income'].describe()
total_income_described

count    2.145400e+04
mean     1.653196e+05
std      9.818730e+04
min      2.066700e+04
25%      1.076230e+05
50%      1.425940e+05
75%      1.958202e+05
max      2.265604e+06
Name: total_income, dtype: float64

Теперь напишем и применим функцию, которая принимает на вход доход и возвращает id в зависимости от принадлежности к квартилю:

In [52]:
def total_income_id(total_income):
    if total_income < total_income_described['25%']:
        return 0
    elif total_income < total_income_described['50%']:
        return 1
    elif total_income < total_income_described['75%']:
        return 2
    else:
        return 3

In [53]:
data['total_income_id'] = data['total_income'].apply(total_income_id)
data.head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,dob_years_id,purpose_id,children_id,total_income_id
0,1,-8437.673028,42.0,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья,1,0,1,3
1,1,-4024.803754,36.0,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля,1,1,1,1
2,0,-5623.422610,33.0,среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья,0,0,0,2
3,3,-4124.747207,32.0,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование,0,2,1,3
4,0,340266.072047,53.0,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу,1,3,0,2


Сформулируем очередной вывод:

In [54]:
print(f'Первый квартиль составлют заёмщики с доходом до {total_income_described["25%"]:.0f} у.е. в месяц')
print(f'Второй квартиль составлют заёмщики с доходом от {total_income_described["25%"]:.0f} у.е. до {total_income_described["50%"]:.0f} у.е. в месяц')
print(f'Третий квартиль составлют заёмщики с доходом от {total_income_described["50%"]:.0f} у.е. до {total_income_described["75%"]:.0f} у.е. в месяц')
print(f'Четвертый квартиль составлют заёмщики с доходом от {total_income_described["75%"]:.0f} у.е. до {total_income_described["max"]:.0f} у.е. в месяц')

Первый квартиль составлют заёмщики с доходом до 107623 у.е. в месяц
Второй квартиль составлют заёмщики с доходом от 107623 у.е. до 142594 у.е. в месяц
Третий квартиль составлют заёмщики с доходом от 142594 у.е. до 195820 у.е. в месяц
Четвертый квартиль составлют заёмщики с доходом от 195820 у.е. до 2265604 у.е. в месяц


## Шаг 3. Ответьте на вопросы

- Есть ли зависимость между наличием детей и возвратом кредита в срок?

У нас достаточно данных для подсчета доли должников среди заёмщиков с детьми и без детей. Напишем функцию, которая возвращает сводную таблицу, котора опишет распредление должников среди групп заёмщиков, определяемыми параметром factor:

In [55]:
def how_it_affects(factor):
    data_pivot = data.pivot_table(index = [factor], columns = 'debt', values = 'gender', aggfunc = 'count')
    data_pivot['ratio'] = round(data_pivot[1] / (data_pivot[0] + data_pivot[1]), 3)
    return data_pivot.sort_values('ratio', ascending = False)

Применим функцию чтобы узнать как наличие детей у заёмщика влияет на возврат кредита:

In [56]:
how_it_affects('children_id')

debt,0,1,ratio
children_id,,,
1,6685,678,0.092
0,13028,1063,0.075


**Вывод**

Должников среди заёмщиков с детьми:  9,2%
<br>
Должников среди заёмщиков без детей:  7,5%
<br>
Вывод не самый очевидный: наличие у заёмщика детей повышает для него вероятность просрочить возврат долга на 1,7%! Очевидно, это связано с тем, что такие заёмщики прежде всего потратят свои деньги на ребёнка, а не на платеж по кредиту.

- Есть ли зависимость между семейным положением и возвратом кредита в срок?

In [57]:
how_it_affects('family_status')

debt,0,1,ratio
family_status,,,
Не женат / не замужем,2536,274,0.098
гражданский брак,3763,388,0.093
женат / замужем,11408,931,0.075
в разводе,1110,85,0.071
вдовец / вдова,896,63,0.066


**Вывод**

Должников среди заёмщиков  не в браке: 9,8%
<br>
Должников среди заёмщиков в гражданском браке: 9,3%
<br>
Должников среди заёмщиков в зарегистрированном браке: 7,5%
<br>
Должников среди заёмщиков в разводе: 7,1%
<br>
Должников среди заёмщиков-вдовцов: 6,6%
<br>
Если заёмщик хотя бы раз был в зарегистрированном браке, то это снижает вероятность просрочек по кредиту на 1,8-3,2%. Возможно это связано с тем, что жизнь в браке учит ответственнее подходить к ведению бюджета.

- Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

In [58]:
how_it_affects('total_income_id')

debt,0,1,ratio
total_income_id,,,
1,4024,385,0.087
2,5771,546,0.086
0,4937,427,0.080
3,4981,383,0.071


**Вывод**

Должников среди заёмщиков с доходом до 107623 у.е. в месяц: 8,0%
<br>
Должников среди заёмщиков с доходом от 107623 до 142594 у.е. в месяц: 8,7%
<br>
Должников среди заёмщиков с доходом от 142594 до 195820 у.е. в месяц: 8,6%
<br>
Должников среди заёмщиков с доходом от 195820 до 2265604 у.е. в месяц: 7,1%
<br>
Снова неочевидный вывод. Заёмщики с самыми низкими доходами не являются самыми ненадежными! При этом шансы стать должниками для групп с доходом от 107623 до 142594 и доходом от 142594 до 195820 практически не отличаются и составляют 8,6-8,7%.

- Как разные цели кредита влияют на его возврат в срок?

In [59]:
how_it_affects('purpose_id')

debt,0,1,ratio
purpose_id,,,
1,3903,403,0.094
2,3643,370,0.092
3,2138,186,0.080
0,10029,782,0.072


**Вывод**

Должников среди заёмщиков с целью "недвижимость": 7,2%
<br>
Должников среди заёмщиков с целью "автомобиль": 9,4%
<br>
Должников среди заёмщиков с целью "образование": 9,2%
<br>
Должников среди заёмщиков с целью "свадьба": 8,0%
<br>
Что можно сказать. Заёмщики заинтересованы платить по кредиту за жильё, должников среди них на 0,8-2,2% меньше чем в других групах. Хуже всего себя проявляется заёмщики, предпочитающие брать в кредит автомобили. 

## Шаг 4. Общий вывод

Исходя из промежуточных выводов идеальный заёмщик выглядит так:
- хотя бы раз состоял в зарегистрированном браке
- не имеет детей
- цель кредит - приобретение жилья
- доход от 195820 у.е. в месяц

Вывод тривиальный: опыт жизни в официальном браке, высокий доход, отсутствие иждевенцев и цель в виде собственного жилья минимизируют шансы клиента банка оказаться среди должников. 